Extração dos dados

In [1]:
#Biblioteca pandera utilizada para criar um schema do df
!pip install pandera
import pandas as pd
import pandera as pa 

In [2]:
#Criar dataframe a partir de um csv
#parse_dates é um parametro que recebe uma lista de séries do df e recebe como o typo datatime
df = pd.read_csv('dados_brutos\ocorrencia.csv', sep = ';', parse_dates=['ocorrencia_dia'], dayfirst=True, encoding='latin-1')
df.head(3)

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,52242,52242,52242,52242,52242,INCIDENTE,NaN,NaN,PORTO ALEGRE,RS,...,2012-01-05,20:27:00,***,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,45331,45331,45331,45331,45331,ACIDENTE,-23.4355555556,-46.4730555556,GUARULHOS,SP,...,2012-01-06,13:44:00,SIM,FINALIZADA,A-582/CENIPA/2014,SIM,2016-09-01,3,1,NÃO
2,45333,45333,45333,45333,45333,ACIDENTE,***,***,VIAMÃO,RS,...,2012-01-06,13:00:00,NaN,FINALIZADA,A-070/CENIPA/2013,SIM,2013-11-27,0,1,NÃO


In [3]:
#Excluir colunas que não serão utilizadas no projeto
df.drop(['codigo_ocorrencia1','codigo_ocorrencia3','codigo_ocorrencia4','ocorrencia_latitude','ocorrencia_longitude','ocorrencia_pais','investigacao_aeronave_liberada','investigacao_status','divulgacao_relatorio_numero','divulgacao_relatorio_publicado','divulgacao_dia_publicacao','total_aeronaves_envolvidas','ocorrencia_saida_pista'], axis=1, inplace=True)
df.head(3)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0


In [4]:
#nullable -> permite valores nulos
#pa.Check.str_length(X,Y) onde x é o valor mínimo e y é o valor máximo de caracteres
#required=False -> coluna não é obrigatória
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia':pa.Column(pa.Int),
        'codigo_ocorrencia2':pa.Column(pa.Int),
        'ocorrencia_classificacao':pa.Column(pa.String),
        'ocorrencia_cidade':pa.Column(pa.String),
        'ocorrencia_uf':pa.Column(pa.String, pa.Check.str_length(2,2)),
        'ocorrencia_aerodromo':pa.Column(pa.String),
        'ocorrencia_dia':pa.Column(pa.DateTime),
        'ocorrencia_hora':pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        'total_recomendacoes':pa.Column(pa.Int)
})

In [5]:
#Validar o df conforme o schema criado anteriormente pelo pandera
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06,16:30:00,0
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,ACIDENTE,JATAÍ,GO,####,2021-12-30,20:30:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,*****,2021-12-31,09:30:00,0
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0


In [6]:
#Verificar os tipos de cada série
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [7]:
#Selecionar apenas o mês de uma coluna do tipo datetime
df.ocorrencia_dia.dt.month

0        1
1        1
2        1
3        1
4        1
        ..
5162    12
5163    12
5164    12
5165    12
5166    12
Name: ocorrencia_dia, Length: 5167, dtype: int64

Limpeza de Dados

In [8]:
#df.loc[x,y] sendo x label e y coluna
df.loc[1,'ocorrencia_cidade']

'GUARULHOS'

In [9]:
'''#Criar índice a partir de uma coluna
df.set_index('codigo_ocorrencia', inplace=True)
df.head(3)'''

"#Criar índice a partir de uma coluna\ndf.set_index('codigo_ocorrencia', inplace=True)\ndf.head(3)"

In [10]:
'''#Excluir o índice criado
df.reset_index(drop=True, inplace=True)
df.head(3)'''

'#Excluir o índice criado\ndf.reset_index(drop=True, inplace=True)\ndf.head(3)'

In [11]:
#Quando ocorrencia_uf = SP, alterar a coluna ocorrencia_classificacao para GRAVE
#df.loc[df.ocorrencia_uf == "SP", ['ocorrencia_classificacao']] = "GRAVE"''

In [12]:
#Tratamento da coluna ocorrencia_uf atribuindo valores nulos (NA) através do pandas
df.loc[df.ocorrencia_aerodromo == '****', ['ocorrencia_aerodromo']] = pd.NA
df.head(3)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,<NA>,2012-01-06,13:00:00,0


In [13]:
#Tratamento com replace para atribuir valores nulos
df.replace(['**','###!','####',"*****",'NULL'], pd.NA, inplace=True)
df.head(3)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,<NA>,2012-01-06,13:00:00,0


In [14]:
#Verificar os valores não informados no dataframe e soma por coluna
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        1905
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [15]:
df.isnull().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        1905
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [16]:
#df.fillna(X) -> Adiciona um valor X em todos os valores nulos do df
#df.fillna(value = {'coluna':X}) -> adiciona um valor X nos valores nulos da coluna
df.fillna(0, inplace = True)
df.head(3)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,0,2012-01-06,13:00:00,0


In [17]:
#Criar uma nova coluna
df['total_recomendacoes_bk'] =df.total_recomendacoes
df.head(3)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,total_recomendacoes_bk
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,0,2012-01-06,13:00:00,0,0


In [18]:
#Excluir uma coluna
df.drop(['total_recomendacoes_bk'], axis=1, inplace=True)
df.head(3)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,0,2012-01-06,13:00:00,0


In [19]:
#Excluir linhas que possuem valores nulos
df.dropna()
df.head(3)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,0,2012-01-06,13:00:00,0


In [20]:
#Excluir valores duplicados
df.drop_duplicates(inplace=True)
df.head(3)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,0,2012-01-06,13:00:00,0
